In [ ]:
import numpy as np
import statsmodels.api as sm
import math
import pandas as pd
from quantopian.research import run_pipeline, returns, get_pricing
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.builtin import EquityPricing
from quantopian.pipeline.factors import CustomFactor, Returns, PercentChange, SimpleMovingAverage
from quantopian.pipeline.filters import QTradableStocksUS
from quantopian.pipeline.data import Fundamentals
from quantopian.pipeline.data.factset import Fundamentals as fctstF
from quantopian.pipeline.data.psychsignal import stocktwits
from quantopian.pipeline import factors, filters, classifiers
import alphalens
from scipy.stats.mstats import winsorize
from sklearn import preprocessing
from scipy.stats.mstats import gmean
from zipline.utils.numpy_utils import (
    repeat_first_axis,
    repeat_last_axis,
)
from quantopian.pipeline.domain import (
    JP_EQUITIES)

class FilterExtend(CustomFactor):
    def compute(self, today, asset_ids, out, values):
        out[:] = np.max(values, axis=0)

class MinVolume(CustomFactor):
    inputs=[EquityPricing.volume]
    window_length=10
    def compute(self, today, asset_ids, out, values):
        # Calculates the column-wise standard deviation, ignoring NaNs
        out[:] = np.min(values, axis=0)

# Create a volume and price filter that filters for stocks in the top 30%.
# We multiply by price to rule out penny stocks that trade in huge volume.
volume_min = MinVolume()
price = EquityPricing.close.latest
univ_filter = ((price * volume_min).percentile_between(70, 100, mask=(volume_min > 0)))


def backshift_returns_series(series, N):
    """Shift a multi-indexed series backwards by N observations in the first level.
    
    This can be used to convert backward-looking returns into a forward-returns series.
    """
    ix = series.index
    dates, sids = ix.levels
    date_labels, sid_labels = map(np.array, ix.labels)
    # Output date labels will contain the all but the last N dates.
    new_dates = dates[:-N]
    # Output data will remove the first M rows, where M is the index of the
    # last record with one of the first N dates.
    cutoff = date_labels.searchsorted(N)
    new_date_labels = date_labels[cutoff:] - N
    new_sid_labels = sid_labels[cutoff:]
    new_values = series.values[cutoff:]
    assert new_date_labels[0] == 0
    new_index = pd.MultiIndex(
        levels=[new_dates, sids],
        labels=[new_date_labels, new_sid_labels],
        sortorder=1,
        names=ix.names,
    )
    return pd.Series(data=new_values, index=new_index)


#Factors taken from https://www.quantopian.com/posts/alpha-combination-via-clustering
WIN_LIMIT = 0

def signalize(df):
    z = (df.rank() - 0.5)/df.count()
    return z.replace(np.nan, z.mean())

def preprocess(a):
    a = a.astype(np.float64)
    a[np.isinf(a)] = np.nan
    a = np.nan_to_num(a - np.nanmean(a))
    a = winsorize(a, limits=[WIN_LIMIT,WIN_LIMIT])
    
    return preprocessing.scale(a)



In [ ]:
quarter_length = 74

class Volatility(CustomFactor):  
    inputs = [EquityPricing.close]  
    window_length = 22  
    def compute(self, today, assets, out, close):  
        # [0:-1] is needed to remove last close since diff is one element shorter  
        daily_returns = np.diff(close, axis = 0) / close[0:-1]  
        out[:] = daily_returns.std(axis = 0) * math.sqrt(252)
        
class MarketCap(CustomFactor):
    # Pre-declare inputs and window_length
    inputs = [fctstF.mkt_val]
    window_length = 1
    
    # Compute market cap value
    def compute(self, today, assets, out, val):
        out[:] = val[-1]
                
class fcf(CustomFactor):
    inputs = [fctstF.pfcf_qf]
    window_length = 1
    window_safe = True
    def compute(self, today, assets, out, fcf_yield):
        out[:] = fcf_yield[-1,:]

class FCFStability(CustomFactor):
    inputs = [fctstF.pfcf_qf]
    window_length = quarter_length*8
    window_safe = True
    def compute(self, today, assets, out, fcf_yield):
        std = np.std([fcf_yield[-1], 
            fcf_yield[-quarter_length], 
            fcf_yield[-quarter_length*2],
            fcf_yield[-quarter_length*3],
            fcf_yield[-quarter_length*4],
            fcf_yield[-quarter_length*5],
            fcf_yield[-quarter_length*6],
            fcf_yield[-quarter_length*7]], axis=0)

        out[:] = -std
        
class Direction(CustomFactor):
    inputs = [EquityPricing.open, EquityPricing.close]
    window_length = 21
    window_safe = True
    def compute(self, today, assets, out, open, close):
        p = (close-open)/close
        out[:] = np.nansum(-p,axis=0)

class mean_rev(CustomFactor):   
    inputs = [EquityPricing.high,EquityPricing.low,EquityPricing.close]
    window_length = 30
    window_safe = True
    def compute(self, today, assets, out, high, low, close):

        p = (high+low+close)/3

        m = len(close[0,:])
        n = len(close[:,0])
        b = np.zeros(m)
        a = np.zeros(m)

        for k in range(10,n+1):
            price_rel = np.nanmean(p[-k:,:],axis=0)/p[-1,:]
            wt = np.nansum(price_rel)
            b += wt*price_rel
            price_rel = 1.0/price_rel
            wt = np.nansum(price_rel)
            a += wt*price_rel

        out[:] = b-a

class volatility(CustomFactor):
    inputs = [EquityPricing.high, EquityPricing.low, EquityPricing.close, EquityPricing.volume]
    window_length = 5
    window_safe = True
    def compute(self, today, assets, out, high, low, close, volume):
        vol = np.nansum(volume,axis=0)*np.nansum(np.absolute((high-low)/close),axis=0)
        out[:] = preprocess(-vol)

class peg_ratio(CustomFactor):
    inputs = [fctstF.pe_qf, fctstF.eps_basic_gr_qf]
    window_length = 1
    window_safe = True
    def compute(self, today, assets, out, pe, e_growth):
        peg_ratio = pe/e_growth
        out[:] = -1.0/peg_ratio[-1,:]

        
class peg_ratio_stability(CustomFactor):
    inputs = [fctstF.pe_qf, fctstF.eps_basic_gr_qf]
    window_length = quarter_length*8
    window_safe = True
    def compute(self, today, assets, out, pe, e_growth):
        var = pe/e_growth
        std = np.std([1/var[-1], 
            1/var[-quarter_length], 
            1/var[-quarter_length*2],
            1/var[-quarter_length*3],
            1/var[-quarter_length*4],
            1/var[-quarter_length*5],
            1/var[-quarter_length*6],
            1/var[-quarter_length*7]], axis=0)

        out[:] = std

        
class SalesGrowth(CustomFactor):
    inputs = [fctstF.sales_gr_qf]
    window_length = 2*252
    window_safe = True
    def compute(self, today, assets, out, sales_growth):
        sales_growth = np.nan_to_num(sales_growth)
        sales_growth = preprocessing.scale(sales_growth,axis=0)
        out[:] = sales_growth[-1]

class SalesGrowthStability(CustomFactor):
    inputs = [fctstF.sales_gr_qf]
    window_length = quarter_length*8
    window_safe = True
    def compute(self, today, assets, out, var):
        mean = np.mean([var[-1], 
            var[-quarter_length], 
            var[-quarter_length*2],
            var[-quarter_length*3],
            var[-quarter_length*4],
            var[-quarter_length*5],
            var[-quarter_length*6],
            var[-quarter_length*7]], axis=0)

        std = np.std([var[-1], 
            var[-quarter_length], 
            var[-quarter_length*2],
            var[-quarter_length*3],
            var[-quarter_length*4],
            var[-quarter_length*5],
            var[-quarter_length*6],
            var[-quarter_length*7]], axis=0)
        
        out[:] = mean/std

        
class GrossMarginChange(CustomFactor):
    window_length = 2*252
    window_safe = True
    inputs = [fctstF.ebit_oper_mgn_qf]
    def compute(self, today, assets, out, ebit_oper_mgn):
#         ebit_oper_mgn = np.nan_to_num(ebit_oper_mgn)
#         ebit_oper_mgn = preprocessing.scale(ebit_oper_mgn,axis=0)
        out[:] = ebit_oper_mgn[-1]

class GrossMarginStability(CustomFactor):
    window_length = quarter_length*8
    window_safe = True
    inputs = [fctstF.ebit_oper_mgn_qf]
    def compute(self, today, assets, out, var):
        var = np.nan_to_num(var)
        mean = np.mean([var[-1], 
            var[-quarter_length], 
            var[-quarter_length*2],
            var[-quarter_length*3],
            var[-quarter_length*4],
            var[-quarter_length*5],
            var[-quarter_length*6],
            var[-quarter_length*7]], axis=0)
        
        std = np.std([var[-1], 
            var[-quarter_length], 
            var[-quarter_length*2],
            var[-quarter_length*3],
            var[-quarter_length*4],
            var[-quarter_length*5],
            var[-quarter_length*6],
            var[-quarter_length*7]], axis=0)
        
        out[:] = mean/std
        
        
base_universe = univ_filter #QTradableStocksUS()

mkt_cap = MarketCap(mask = base_universe)
vol = Volatility(mask = base_universe)

alpha1 = fcf(mask = base_universe) 
alpha2 = Direction(mask = base_universe)        
alpha3 = mean_rev(mask = base_universe)  
alpha4 = volatility(mask = base_universe)   
alpha5 = FCFStability(mask = base_universe)
alpha6 = peg_ratio(mask = base_universe)        
alpha7 = peg_ratio_stability(mask = base_universe)

alpha8 = SalesGrowth(mask = base_universe)        
alpha9 = SalesGrowthStability(mask = base_universe)        

alpha10 = GrossMarginChange(mask = base_universe)  
alpha11 = GrossMarginStability(mask = base_universe)


# alpha12 = Gross_Income_Margin(mask = base_universe)        
# alpha13 = MaxGap(mask = base_universe)        
# alpha14 = CapEx_Vol(mask = base_universe)  

# alpha15 = fcf_ev(mask = base_universe)        
# alpha16 = DebtToTotalAssets(mask = base_universe)        
# alpha17 = TEM(mask = base_universe)        
# alpha18 = Piotroski(mask = base_universe)        
# alpha19 = Altman_Z(mask = base_universe)        
# alpha20 = Quick_Ratio(mask = base_universe)        
# alpha21 = AdvancedMomentum(mask = base_universe)        
# alpha22 = STA(mask = base_universe)        
# alpha23 = SNOA(mask = base_universe)        
# alpha24 = ROA(mask = base_universe)        
# alpha25 = FCFTA(mask = base_universe)        
# alpha26 = ROA_GROWTH(mask = base_universe)        
# alpha27 = FCFTA_ROA(mask = base_universe)        
# alpha28 = FCFTA_GROWTH(mask = base_universe)        
# alpha29 = LTD_GROWTH(mask = base_universe)        
# alpha30 = CR_GROWTH(mask = base_universe)        
# alpha31 = GM_GROWTH(mask = base_universe) 

# alpha32 = ATR_GROWTH(mask = base_universe) 
# alpha33 = NEQISS(mask = base_universe) 
# alpha34 = GM_GROWTH_2YR(mask = base_universe) 
# alpha35 = GM_STABILITY_2YR(mask = base_universe) 

# alpha36 = ROA_GROWTH_2YR(mask = base_universe) 
# alpha37 = ROIC_GROWTH_2YR(mask = base_universe) 
# alpha38 = GM_GROWTH_8YR(mask = base_universe) 
# alpha39 = GM_STABILITY_8YR(mask = base_universe) 
# alpha40 = ROA_GROWTH_8YR(mask = base_universe) 
# alpha41 = ROIC_GROWTH_8YR(mask = base_universe) 
# alpha42 = Quick_Ratio_Stability(mask = base_universe)
# alpha43 = FCFStability(mask = base_universe)

# alpha44 = growthscorestability(mask = base_universe)


# alpha48 = Gross_Income_Margin_Stability(mask = base_universe)
# alpha49 = CapEx_Vol_Stability(mask = base_universe)
# alpha50 = fcf_ev_stability(mask = base_universe)
# alpha51 = DebtToTotalAssetsStability(mask = base_universe)
# alpha52 = fcf_growth_mean(mask = base_universe)

# alpha53 = dta_growth_mean(mask = base_universe)
# alpha54 = TEM_GROWTH(mask = base_universe)
# alpha55 = Altman_Z_Stability(mask = base_universe)

# alpha56 = Quick_Ratio_Mean(mask = base_universe)
# alpha57 = Quick_Ratio_Std(mask = base_universe)
# alpha58 = Quick_Ratio_Stability(mask = base_universe)
# alpha59 = STA_Stability(mask = base_universe)
# alpha60 = STA_Mean(mask = base_universe)
# alpha61 = STA_Std(mask = base_universe)
# alpha62 = ROA_Mean(mask = base_universe)
# alpha63 = ROA_Std(mask = base_universe)
# alpha64 = ROA_Stability(mask = base_universe)
# alpha65 = Current_Ratio_Mean(mask = base_universe)
# alpha66 = Current_Ratio_Std(mask = base_universe)

# alpha67 = Current_Ratio_Stability(mask = base_universe)
# alpha68 = ATR_GROWTH_MEAN(mask = base_universe)
# alpha69 = ATR_GROWTH_STD(mask = base_universe)
# alpha70 = ATR_GROWTH_STABILITY(mask = base_universe)
# alpha71 = pcf_ratio_stability(mask = base_universe)
# alpha72 = ps_ratio_stability(mask = base_universe)
# alpha73 = fey_stability(mask = base_universe)

# alpha74 = ey_stability(mask = base_universe)
# alpha75 = sy_stability(mask = base_universe)
# alpha76 = revenue_growth_mean(mask = base_universe)
# alpha77 = revenue_growth_std(mask = base_universe)

# alpha78 = revenue_growth_stability(mask = base_universe)
# alpha79 = ebitda_margin_mean(mask = base_universe)
# alpha80 = ebitda_margin_std(mask = base_universe)
# alpha81 = ebitda_margin_stability(mask = base_universe)
# alpha82 = pretax_margin_mean(mask = base_universe)

# alpha83 = pretax_margin_std(mask = base_universe)

# alpha84 = pretax_margin_stability(mask = base_universe)

start_date = '2015-06-01'
end_date = '2018-09-01'
domain = JP_EQUITIES

calendar = domain.calendar
# Roll input dates to the next trading session.
start_date = calendar.minute_to_session_label(pd.Timestamp(start_date, tz='UTC'))
end_date = calendar.minute_to_session_label(pd.Timestamp(end_date, tz='UTC'))

# if factor_screen is None:
#     factor_screen = factor.notnull()

# # Run pipeline to get factor values and quantiles.
# factor_pipe = Pipeline(
#     {'factor': factor, 
#      'factor_quantile': factor.quantiles(quantiles, mask=factor_screen)},
#     screen=factor_screen,
#     domain=domain,
# )

pipe_alpha_factors = Pipeline(
    columns={
        'vol': vol,
        'mkt_cap':mkt_cap,
        
'alpha1':alpha1,
'alpha2':alpha2,
'alpha3':alpha3,
'alpha4':alpha4,
'alpha5':alpha5,
'alpha6':alpha6,
'alpha7':alpha7,
        
'alpha8': alpha8,
'alpha9': alpha9,
        
'alpha10': alpha10,
'alpha11': alpha11,
        

        
        
# 'alpha8':alpha8,
# 'alpha9':alpha9,
# 'alpha10':alpha10,
# 'alpha11':alpha11,
# 'alpha12':alpha12,
# 'alpha13':alpha13,
# 'alpha14':alpha14,
# 'alpha15':alpha15,
# 'alpha16':alpha16,
# 'alpha17':alpha17,
# 'alpha18':alpha18,
# 'alpha19':alpha19,
# 'alpha20':alpha20,
# 'alpha21':alpha21,
# 'alpha22':alpha22,
# 'alpha23':alpha23,
# 'alpha24':alpha24,
# 'alpha25':alpha25,
# 'alpha26':alpha26,
# 'alpha27':alpha27,
# 'alpha28':alpha28,
# 'alpha29':alpha29,
# 'alpha30':alpha30,
# 'alpha31':alpha31,
# 'alpha32':alpha32,
# 'alpha33':alpha33,
# 'alpha34':alpha34,
# 'alpha35':alpha35,
# 'alpha36':alpha36,
# 'alpha37':alpha37,
# 'alpha38':alpha38,
# 'alpha39':alpha39,
# 'alpha40':alpha40,
# 'alpha41':alpha41,
# 'alpha42': alpha42,
# 'alpha43': alpha43,
#         'alpha44':alpha44,
#         'alpha45':alpha45,
#         'alpha46': alpha46,
#         'alpha47': alpha47,
#         'alpha48': alpha48,
#                 'alpha49': alpha49,
#         'alpha50': alpha50,
#         'alpha51': alpha51
#         'alpha52': alpha52,
#         'alpha53': alpha53
#             'alpha54': alpha54,
#         'alpha55': alpha55,
#                 'alpha56': alpha56,
#                 'alpha57': alpha57,
#                 'alpha58': alpha58,
#                         'alpha59': alpha59,
#                 'alpha60': alpha60,
#                 'alpha61': alpha61,
#         'alpha62':alpha62,
#                 'alpha63':alpha63,
#                 'alpha64':alpha64,
#         'alpha65':alpha65,
#                 'alpha66':alpha66,
#                 'alpha67':alpha67
#                         'alpha68':alpha68,
#                         'alpha69':alpha69,
#                         'alpha70':alpha70
#                                 'alpha71':alpha71,
#                 'alpha72':alpha72,
#                 'alpha73':alpha73,
#                 'alpha74':alpha74,
#                 'alpha75':alpha75,
#         'alpha76': alpha76,
#         'alpha77': alpha77,
#         'alpha78': alpha78,
        
#         'alpha79': alpha79,
#         'alpha80': alpha80,
#         'alpha81': alpha81

#                 'alpha82': alpha82,
#         'alpha83': alpha83,
#         'alpha84': alpha84

        
    },
    screen=base_universe,
    domain=domain
)


output = run_pipeline(pipe_alpha_factors, start_date, end_date, chunksize=250)

In [ ]:
# assets = output.index.levels[1].unique()
# We need to get a little more pricing data than the 
# length of our factor so we can compare forward returns.
# We'll tack on another month in this example.
# pricing = get_pricing(assets, start_date='2015-06-01', end_date='2018-09-01', fields='close_price')


In [ ]:
# factor_names = ['alpha1','alpha2','alpha3','alpha4','alpha5','alpha6','alpha7','alpha8','alpha9','alpha10',
# 'alpha11','alpha12','alpha13','alpha14','alpha15','alpha16','alpha17','alpha18','alpha19','alpha20',
# 'alpha21','alpha22','alpha23','alpha24','alpha25','alpha26','alpha27','alpha28','alpha29','alpha30',
# 'alpha31','alpha32','alpha33','alpha34','alpha35','alpha36','alpha37','alpha38','alpha39','alpha40',
# 'alpha41']

# factor_names = ['alpha2','alpha4','alpha11','alpha14','alpha31',
#                 'alpha35', 'alpha43', 'alpha45', 'alpha46', 'alpha52', 
#                 'alpha55', 'alpha66', 'alpha73', 'alpha77', 'alpha82', 'alpha83', 'alpha84']

factor_names = []

for i in range(1,12):
# for i in [10,11]:
    factor_names.append('alpha'+str(i))

def wmean(name):
    return (output['mkt_cap']*output[name]).sum(level=0)/output['mkt_cap'].mean(level=0)

factor_data = pd.DataFrame()
#Demean the alphas
for name in factor_names:
    output['s'+name] = output[name]
    s = signalize(output['s'+name])
    factor_data[name] = s.sub(s.mean(level=0), level=0)
#     output[name] = output[name].sub(wmean(name), level=0)

In [ ]:
# output['vol_rank'] = signalize(output['vol'])
# output['mk_rank'] = signalize(output['mkt_cap']**(1.0/3.0))
# # output['weights'] = 0.3*output['vol_rank'] + 0.7*output['mk_rank']
# # output['weights'] = output['vol_rank'] 
# output['weights'] = output['vol_rank']/output['vol_rank'].sum()

In [ ]:
# returns = pricing.pct_change(periods=5).shift(-5)
# #Demean the returns
# returns = returns.sub(returns.mean(axis=1), axis=0).stack()
# data = pd.concat([returns.rename('returns'), data], axis=1).dropna()

returns_lengths = (1,5,10)

returns_pipe = Pipeline(
        columns={'daily_returns': Returns(window_length=2)}, 
        domain=domain, 
        screen=FilterExtend(
            inputs=[base_universe], 
            window_length=max(returns_lengths)+1
        ).eq(1)
)

returns_start_date = start_date
returns_end_date = end_date + domain.calendar.day * max(returns_lengths)
raw_returns = run_pipeline(returns_pipe, returns_start_date, returns_end_date, chunksize=500)
    
shifted_returns = {}
column_order = []
daily_returns = raw_returns['daily_returns']

for length in returns_lengths:
    # Shift 1-day returns back by a day, 5-day returns back by 5 days, etc.
    colname = '{}D'.format(length)
    column_order.append(colname)
    shifted_returns[colname] = backshift_returns_series(daily_returns, length)
        
# Merge backshifted returns into a single frame indexed like our desired output.
merged_returns = pd.DataFrame(
    data=shifted_returns, 
    index=factor_data.index, 
    columns=column_order,
)

merged_returns = merged_returns.sub(merged_returns.mean(axis=1), axis=0)
#.stack()
    
# print merged_returns.head()        

In [ ]:
# Concat factor results and forward returns column-wise.
data = pd.concat([factor_data, merged_returns], axis=1)
# data.index.set_names(['date', 'asset'], inplace=True)

# Drop NaNs
data = data.dropna(how='any')

# Add a Business Day Offset to the DateTimeIndex
data.index.levels[0].freq = pd.tseries.offsets.BDay()

# print data.head()
data.reset_index(inplace=True)
data = data.rename(columns={'level_0':'dates', 'level_1': 'sid'})

In [ ]:
print factor_names

In [ ]:
significant_factor_names = []

est = sm.OLS(
    100*data[['5D']], 
#         data[significant_factor_names])
    data[['alpha11']])
est2 = est.fit(cov_type='cluster', cov_kwds={'groups': data['dates']})

print est2.summary()

In [ ]:
print output['salpha1']